In [ ]:
! pip install pandas
! pip install openpyxl

import pandas as pd
import rdflib
import hashlib
import numpy as np
from datetime import datetime
from pandas.api.types import CategoricalDtype
from rdflib import Literal, Namespace, RDF, URIRef
from rdflib.namespace import FOAF, XSD
from rdflib import Graph, Namespace, RDF, RDFS, OWL
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

### Step01: Load RDF data from files


In [ ]:
# Load RDF data
rdf_files = [
    "output/RDFoutputCattleSampleLab1.ttl",
    "output/RDFoutputCattleSampleLab2.ttl",
    "output/RDFoutputCattleSampleLab3.ttl",
    "output/RDFoutputCattleSampleLab4.ttl",
    "output/RDFoutputCattleSampleLab5.ttl",
    "output/RDFoutputPigSampleLab1.rdf",
    "output/RDFoutputPigSampleLab2.rdf",
    "output/RDFoutputPigSampleLab3.rdf",
    "output/RDFoutputPigSampleLab4.rdf",
    "output/RDFoutputPigSampleLab5.rdf",
    "output/RDFoutputPoultrySampleLab1.rdf",
    "output/RDFoutputSalmonSampleLab1.rdf",
]

g = Graph()
for rdf_file in rdf_files:
    g.parse(rdf_file, format="turtle")


# If you want to print graph then following is the procedure to print graph

In [ ]:

# Initialize lists to store RDF subjects
rdf_subjects = []

# Iterate through RDF triples and extract subjects
for subject, _, _ in g:
    rdf_subjects.append(subject)

# Convert RDF subjects to DataFrames
data_frames = []
for subject in rdf_subjects:
    rdf_data = g.triples((subject, None, None))
    data = {}
    for _, predicate, obj in rdf_data:
        column_name = predicate.split("#")[-1]
        data[column_name] = [obj.toPython() if isinstance(obj, Literal) else obj]
    df = pd.DataFrame(data)
    data_frames.append(df)

# Concatenate DataFrames
barometer_Tableau = pd.concat(data_frames, ignore_index=True)

# Convert to datetime and add Month and Year columns
barometer_Tableau['http://www.purl.org/decide/LivestockHealthOntohasDate'] = pd.to_datetime(barometer_Tableau['http://www.purl.org/decide/LivestockHealthOntohasDate'])
barometer_Tableau['Month'] = barometer_Tableau['http://www.purl.org/decide/LivestockHealthOntohasDate'].dt.month
barometer_Tableau['Year'] = barometer_Tableau['http://www.purl.org/decide/LivestockHealthOntohasDate'].dt.year


In [ ]:
barometer_Tableau

In [ ]:
# Define filename and folder path for RDF
folder_path_rdf = "output/"
base_filename   = "barometer_combined"
current_date    = datetime.now().strftime("%Y-%m-%d")
new_filename_rdf = f"{folder_path_rdf}{base_filename}_{current_date}.rdf"

# Save RDF graph to file
g.serialize(new_filename_rdf, format="turtle")